# Adição de hino

Proposta de adição de hinos. A ideia é simples:

1. Criar arquivo de hino em Markdown, como se fosse na coletânea, com título como heading, e seções (Coro, Final, etc.) em negrito
2. Com arquivo de entrada, gerar SQL compatível com a estrutura do banco de dados, e adicionar a arquivo de migração

## 1. Leitura dos arquivos

Passo a passo

In [57]:
from glob import glob

entrada = glob("arquivos_hinos\\*.md")
entrada

['arquivos_hinos\\ABRE O CORAÇÃO.md',
 'arquivos_hinos\\PAI REUNIDOS ESTAMOS.md']

In [35]:
with open(entrada[0], "r", encoding="utf-8") as f:
    conteudo = f.readlines()

conteudo

['# ABRE O CORAÇÃO\n',
 '\n',
 'ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR,\n',
 'E MODIFICAR PRA SEMPRE O TEU VIVER,\n',
 'E NESTE LOUVOR DEUS VAI SE REVELAR,\n',
 'PARA RESTAURAR TEU CORAÇÃO.\n',
 '\n',
 '**CORO**\n',
 'O SENHOR SÓ QUER DE TI SINCERIDADE,\n',
 '(ELE VÊ TEU CORAÇÃO)\n',
 'PRA PODER AGIR EM TEU FAVOR.\n',
 '(PARA AGIR EM TEU FAVOR)\n',
 'O SENHOR SÓ QUER DE TI SINCERIDADE,\n',
 '(ELE VÊ TEU CORAÇÃO)\n',
 'PARA DERRAMAR O SEU AMOR.\n',
 '\n',
 'ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR,\n',
 '(DEIXA ENTRAR)\n',
 'E MODIFICAR PRA SEMPRE O TEU VIVER.\n',
 '(O TEU VIVER)\n',
 'E NESTE LOUVOR DEUS VAI SE REVELAR,\n',
 '(SE REVELAR)\n',
 'PARA RESTAURAR TEU CORAÇÃO.\n',
 '\n',
 '**CORO (2X)**\n',
 'O SENHOR SÓ QUER DE TI SINCERIDADE,\n',
 '(ELE VÊ TEU CORAÇÃO)\n',
 'PRA PODER AGIR EM TEU FAVOR.\n',
 '(PARA AGIR EM TEU FAVOR)\n',
 'O SENHOR SÓ QUER DE TI SINCERIDADE,\n',
 '(ELE VÊ TEU CORAÇÃO)\n',
 'PARA DERRAMAR O SEU AMOR.\n']

In [36]:
titulo = conteudo[0].replace("# ", "").strip()
titulo

'ABRE O CORAÇÃO'

In [37]:
texto = conteudo[1:]
texto_full = []
for linha in texto:
    if linha.startswith("**"):
        texto_full.append(f"<b>{linha.split('**')[1].strip()}</b>\n")
    else:
        texto_full.append(linha)

texto_full = "".join(texto_full).strip()
print(texto_full)

ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR,
E MODIFICAR PRA SEMPRE O TEU VIVER,
E NESTE LOUVOR DEUS VAI SE REVELAR,
PARA RESTAURAR TEU CORAÇÃO.

<b>CORO</b>
O SENHOR SÓ QUER DE TI SINCERIDADE,
(ELE VÊ TEU CORAÇÃO)
PRA PODER AGIR EM TEU FAVOR.
(PARA AGIR EM TEU FAVOR)
O SENHOR SÓ QUER DE TI SINCERIDADE,
(ELE VÊ TEU CORAÇÃO)
PARA DERRAMAR O SEU AMOR.

ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR,
(DEIXA ENTRAR)
E MODIFICAR PRA SEMPRE O TEU VIVER.
(O TEU VIVER)
E NESTE LOUVOR DEUS VAI SE REVELAR,
(SE REVELAR)
PARA RESTAURAR TEU CORAÇÃO.

<b>CORO (2X)</b>
O SENHOR SÓ QUER DE TI SINCERIDADE,
(ELE VÊ TEU CORAÇÃO)
PRA PODER AGIR EM TEU FAVOR.
(PARA AGIR EM TEU FAVOR)
O SENHOR SÓ QUER DE TI SINCERIDADE,
(ELE VÊ TEU CORAÇÃO)
PARA DERRAMAR O SEU AMOR.


In [46]:
import re

TAGS_LITERAIS = ["TODOS", "M", "H", "T", "BIS", "VARÕES", "SERVAS", "CORO "]
TAGS_CONTROLE = [
    "ÍNDICE",
    "CORO (2X)",
    "\n\nCORO",
    "CORO\n",
    "1X",
    "2X",
    "3X",
    "4X",
    "()",
    "(TODOS)",
    "(M)",
    "(H)",
    "(T)",
    "(BIS)",
    "(VARÕES)",
    "(SERVAS)",
    "REPETIR O LOUVOR",
    "REPETIR 1ª ESTROFE",
    "FINAL:",
    "BIS NO FINAL",
    "IGREJA CRISTÃ MARANATA",
    "ATUALIZAÇÃO",
    "\nINSTRUMENTOS",
]


def set_text_clean(texto):
    texts_clean = [re.sub(r"\s+", " ", text) for text in texto]
    texts_clean = [line.replace("**", "") for line in texts_clean]
    texts_clean = [line for line in texts_clean if line not in TAGS_LITERAIS]

    new_texts_clean = []
    for line in texts_clean:
        for tag in TAGS_CONTROLE:
            line = line.upper().replace(tag, "")
        line = line.strip()
        if line:
            new_texts_clean.append(line)
    texts_clean = new_texts_clean
    # texts_clean = list(dict.fromkeys(texts_clean))
    texts_clean = " ".join(texts_clean)
    texts_clean = texts_clean.replace("\n", " ")
    # remove all double quotes
    texts_clean = texts_clean.replace("“", "")
    texts_clean = texts_clean.replace("”", "")
    texts_clean = texts_clean.replace('"', "")
    return texts_clean


texto_clean = set_text_clean(texto)
texto_clean

'ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR, E MODIFICAR PRA SEMPRE O TEU VIVER, E NESTE LOUVOR DEUS VAI SE REVELAR, PARA RESTAURAR TEU CORAÇÃO. O SENHOR SÓ QUER DE TI SINCERIDADE, (ELE VÊ TEU CORAÇÃO) PRA PODER AGIR EM TEU FAVOR. (PARA AGIR EM TEU FAVOR) O SENHOR SÓ QUER DE TI SINCERIDADE, (ELE VÊ TEU CORAÇÃO) PARA DERRAMAR O SEU AMOR. ABRE O CORAÇÃO, DEIXA O SENHOR ENTRAR, (DEIXA ENTRAR) E MODIFICAR PRA SEMPRE O TEU VIVER. (O TEU VIVER) E NESTE LOUVOR DEUS VAI SE REVELAR, (SE REVELAR) PARA RESTAURAR TEU CORAÇÃO. O SENHOR SÓ QUER DE TI SINCERIDADE, (ELE VÊ TEU CORAÇÃO) PRA PODER AGIR EM TEU FAVOR. (PARA AGIR EM TEU FAVOR) O SENHOR SÓ QUER DE TI SINCERIDADE, (ELE VÊ TEU CORAÇÃO) PARA DERRAMAR O SEU AMOR.'

### Função

In [47]:
def processa_texto(arquivo):
    with open(arquivo, "r", encoding="utf-8") as f:
        conteudo = f.readlines()

    titulo = conteudo[0].replace("# ", "").strip()

    texto = conteudo[1:]
    texto_clean = set_text_clean(texto)

    texto_full = []
    for linha in texto:
        if linha.startswith("**"):
            texto_full.append(f"<b>{linha.split('**')[1].strip()}</b>\n")
        else:
            texto_full.append(linha)

    texto_full = "".join(texto_full).strip()

    return titulo, texto_full, texto_clean

## 2. Processamento do texto em SQL

In [56]:
arquivo_migracao = "010-add-hinos-avulsos.sql"

with open("..\\db\\migrations\\" + arquivo_migracao, "w", encoding="utf-8") as f:
    for arquivo in entrada:
        titulo, texto_full, texto_clean = processa_texto(arquivo)

        linha = (
            "INSERT INTO hino (numero, nome, texto, texto_limpo, coletanea_id, date_insert, date_update) VALUES ('null','"
            + titulo
            + "','"
            + texto_full.replace("\n", "\\n").replace("'", "''")
            + "','"
            + texto_clean.replace("'", "''")
            + "',4,CURRENT_TIMESTAMP,CURRENT_TIMESTAMP);\n"
        )

        f.write(linha)